In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
from statistics import mean

sns.set_theme()

In [2]:
G = nx.read_gml('patients_gml/P19.gml')

Mixing Score - $\frac{immune-to-tumor}{immune-to-immune}$

patients with less than 250 immune cells - cold

patients with mixing score < 0.22 - compartmentalized

patients with mixing score > 0.22 - mixed

In [3]:
# Illustration required

def get_mixing_score(G):
    number_of_immune_cells = sum(1 for node in G.nodes() if G.nodes[node]['type'] in ['Helper T cell', 'Killer T cell', 'T cell', 'B cell', 'Macrophage'])

    if number_of_immune_cells < 250:
        return -1

    immune_to_tumor = sum(1 for u, v in G.edges() if (G.nodes[u]['type'] == 'Tumor' and G.nodes[v]['type'] in ['Helper T cell', 'Killer T cell', 'T cell', 'B cell', 'Macrophage']) or (G.nodes[v]['type'] == 'Tumor' and G.nodes[u]['type'] in ['Helper T cell', 'Killer T cell', 'T cell', 'B cell', 'Macrophage']))

    immune_to_immune = sum(1 for u, v in G.edges() if (G.nodes[u]['type'] in ['Helper T cell', 'Killer T cell', 'T cell', 'B cell', 'Macrophage'] and G.nodes[v]['type'] in ['Helper T cell', 'Killer T cell', 'T cell', 'B cell', 'Macrophage']))

    return immune_to_tumor / immune_to_immune

In [4]:
mixing_score = get_mixing_score(G)
print(f"Mixing Score: {mixing_score:.2f}")

if mixing_score == -1:
    print('Patient is cold')
elif mixing_score < 0.22:
    print('Patient is compartmentalized')
else:
    print('Patient is mixed')

Mixing Score: -1.00
Patient is cold


Network Centrality Measures used: Network Paper	

1. Degree - Number of edges/nodes - degree distribution
	- Cell type specific node degrees created by separating nodes by cell type
	- Mean degree of each cell type - see if:
		- Stromal cells degree lower than tumor degree - means more interactions of tumor cells with other cells than stromal cell interactions
		- Lymphocytes have more spread out degree distribution - mixture of connection patterns

2. Clustering Coefficient - $\frac{\text{No. of closed triplet}}{\text{No. of all triplets}}$. See if:
	- Clustering coefficient of stromal cells - higher variability than Cancer and immune cells 

3. Stromal Clustering - Average CC of stromal cells in a tumor, only considering the stromal neighbors of stromal cells

4. Stromal Barrier - Count of the number of stromal cells that a lymphocyte has to cross to reach a cancer cluster
	- Overall Stromal Barrier - The average of the individual stromal barriers of lymphocytes in the sample
	- Stromal clustering and Stromal barrier: See if inversely correlated with lymphocyte percentage and positively correlated with stromal percentage

5. Betweenness Centrality - Measures the network flow or traffic in a communication network based on shortest paths
	- For each type of cell, betweenness computed and averaged to represent BC of this cell type in the network

6. Relative percentages of different cell types in each sample

7. See if there are any correlations between the occurrence of different IC subtypes:
	- Do T cells correlate with B cells
	- Do macrophages correlate with endothelium cells

In [5]:
# fig, axs = plt.subplots(2, 3, figsize=(12,8))
# types_of_nodes = [ 'Whole', 'Tumor', 'B cell', 'CD3 (Green #2)', 'CD68 (NarrownRed)', 'Region*']

# for i, ax in enumerate(axs.flat):
#     degree_sequence = []
#     if i == 0:
#         degree_sequence = [d for _, d in G.degree()]
#     else:
#         degree_sequence = [d for n, d in G.degree() if G.nodes[n]['type'] == types_of_nodes[i]]
#     ax.hist(degree_sequence, bins=range(max(degree_sequence) + 2), align='left', rwidth=0.8)
#     ax.set_title(f'{types_of_nodes[i]}')

# plt.show()

In [6]:
stroma_cc = nx.clustering(G, nodes=[node for node in G.nodes() if G.nodes[node]['type'] == 'Other'])

In [7]:
stroma_subgraph = G.subgraph([node for node in G.nodes() if G.nodes[node]['type'] == 'Other'])
avg_stroma_cc = nx.average_clustering(stroma_subgraph, nodes=[node for node in G.nodes() if G.nodes[node]['type'] == 'Other'])
print(f'Average CC of Stroma (only counting stromal neighbours): {avg_stroma_cc:.2f}')

Average CC of Stroma (only counting stromal neighbours): 0.62


Stromal Barrier

In [8]:
b_cells = [node for node, data in G.nodes(data=True) if data.get('type') == 'B cell']
tumor_nodes = [node for node, data in G.nodes(data=True) if data.get('type') == 'Tumor']

In [9]:
shortest_paths = {}
closest_tumor = None
closest_path = 1000000

for b_cell in b_cells:
    shortest_paths[b_cell] = {}
    
    for tumor_node in tumor_nodes:
        try:
            length = nx.shortest_path_length(G, source=b_cell, target=tumor_node)
        except:
            continue

        if length < closest_path and length > 1:
                closest_path = length
                closest_tumor = tumor_node
    
    shortest_paths[b_cell] = {closest_tumor: closest_path}
    print(shortest_paths)

{'2825': {'734': 2}}
{'2825': {'734': 2}, '2826': {'734': 2}}


In [10]:
num_other_nodes = {}
for b_cell, paths in shortest_paths.items():
    num_other_nodes[b_cell] = {}
    for tumor_node, length in paths.items():
        path = nx.shortest_path(G, source=b_cell, target=tumor_node)
        num_other = sum(1 for node in path if G.nodes[node]['type'] == 'Other')   # Subtract 1 to exclude the starting 'B cell'
        num_other_nodes[b_cell][tumor_node] = num_other
# Find average number of 'Other' nodes for all B cells

avg_num_other_nodes = sum(sum(num_other_nodes[b_cell].values()) for b_cell in num_other_nodes) / len(num_other_nodes)

print("Average number of 'Stroma' nodes in the path for all B cells:", avg_num_other_nodes)

Average number of 'Stroma' nodes in the path for all B cells: 0.5


Betweenness Centrality

In [11]:
types_of_nodes = ['Helper T cell', 'Killer T cell', 'T cell', 'B cell', 'Macrophage', 'Other', 'Tumor']
avg_centrality = {}

for node_type in types_of_nodes:
    G_subgraph = G.subgraph(nodes=[node for node, data in G.nodes(data=True) if data.get('type') == node_type])
    betweenness_centrality = nx.betweenness_centrality(G_subgraph)
    avg_centrality[node_type] = mean(betweenness_centrality.values())

In [12]:
print(avg_centrality)

{'Helper T cell': 0.0, 'Killer T cell': 0.0, 'T cell': 0.0, 'B cell': 0.0, 'Macrophage': 0.00010426527280459865, 'Other': 0.009677892749761647, 'Tumor': 0.01699290405167136}
